In [24]:
import os
import glob
import json
import numpy as np
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

## Read CHIRPS json and export all in one netcdf file

In [2]:
dir_base = os.path.join('/','media','arturo','T9','Data','Italy','Satellite','CHIRPS')
dir_json = os.path.join('/','home','arturo','github','downpy','point')

In [3]:
data = xr.open_dataset(os.path.join(dir_base,'1dy','CHIRPS_Italy_1dy_1981_01_01_2024_06_30.nc'))

lat = data.lat.values
lon = data.lon.values
lon2d, lat2d = np.meshgrid(lon, lat)

del data

In [4]:
files_list = glob.glob(os.path.join(dir_json,'*.json'))
files_list = sorted(files_list)
files_len = len(files_list)
print(f'Number of files: {files_len}')

Number of files: 26520


In [5]:
json_read = files_list[0]
with open (json_read) as f:
        param = json.load(f)

In [44]:
NYs_2d = np.zeros([22, len(lat),len(lon)])
CYs_2d = np.zeros([22, len(lat),len(lon)])
WYs_2d = np.zeros([22, len(lat),len(lon)])
MEVs_2d = np.zeros([6, len(lat),len(lon)])

NYd_2d = np.zeros([22, len(lat),len(lon)])
CYd_2d = np.zeros([22, len(lat),len(lon)])
WYd_2d = np.zeros([22, len(lat),len(lon)])
MEVd_2d = np.zeros([6, len(lat),len(lon)])

BETA_2d = np.zeros([len(lat),len(lon)])
GAMMA_2d = np.zeros([len(lat),len(lon)])
FUNVAL_2d = np.zeros([len(lat),len(lon)])

for nn in range(files_len):
    with open(files_list[nn], 'r') as f:
        tmp = json.load(f)
    
    year_ = np.fromstring(tmp['YEARS'].strip('[]'), sep=' ')
    Tr_ = np.fromstring(tmp['Tr'].strip('[]'), sep=' ')
    
    NYs_ = np.fromstring(tmp['NYs'].strip('[]'), sep=' ')
    CYs_ = np.fromstring(tmp['CYs'].strip('[]'), sep=' ')
    WYs_ = np.fromstring(tmp['WYs'].strip('[]'), sep=' ')
    mev_s_ = np.fromstring(tmp['mev_s'].strip('[]'), sep=' ')
    
    NYd_ = np.fromstring(tmp['NYd'].strip('[]'), sep=' ')
    CYd_ = np.fromstring(tmp['CYd'].strip('[]'), sep=' ')
    WYd_ = np.fromstring(tmp['WYd'].strip('[]'), sep=' ')
    mev_d_ = np.fromstring(tmp['mev_d'].strip('[]'), sep=' ')
    
    beta_ = tmp['beta']
    gamma_ = tmp['gam_d']
    
    funval_ = tmp['corr_down_funval']

    lat_pos = np.where(lat==float(tmp['clat']))[0][0]
    lon_pos = np.where(lon==float(tmp['clon']))[0][0]

    NYs_2d[:, lat_pos, lon_pos] = NYs_
    CYs_2d[:, lat_pos, lon_pos] = CYs_
    WYs_2d[:, lat_pos, lon_pos] = WYs_
    MEVs_2d[:,lat_pos, lon_pos] = mev_s_
    
    NYd_2d[:, lat_pos, lon_pos] = NYd_
    CYd_2d[:, lat_pos, lon_pos] = CYd_
    WYd_2d[:, lat_pos, lon_pos] = WYd_
    MEVd_2d[:,lat_pos, lon_pos] = mev_d_

    BETA_2d[lat_pos, lon_pos] = beta_
    GAMMA_2d[lat_pos, lon_pos] = gamma_
    FUNVAL_2d[lat_pos, lon_pos] = funval_

In [55]:
DOWN_xr = xr.Dataset(data_vars={
                    "NYs": (("year","lat","lon"), NYs_2d),
                    "CYs": (("year","lat","lon"), CYs_2d),
                    "WYs": (("year","lat","lon"), WYs_2d),
                    "Mev_s": (("Tr","lat","lon"), MEVs_2d),
                    "NYd": (("year","lat","lon"), NYd_2d),
                    "CYd": (("year","lat","lon"), CYd_2d),
                    "WYd": (("year","lat","lon"), WYd_2d),
                    "Mev_d": (("Tr","lat","lon"), MEVd_2d),
                    "BETA": (("lat","lon"), BETA_2d),
                    "GAMMA": (("lat","lon"), GAMMA_2d),
                    "FUNVAL": (("lat","lon"), FUNVAL_2d)
                    },
        coords={'year':year_,'Tr':Tr_,'lat': lat, 'lon': lon},
        attrs=dict(description=f"""Downscaling for CHIRPS in the Italy area bounded by 
                    longitudes 6.5 to 19 and latitudes 36.5 to 48, using mar as the acf function, 1 mm' 
                    threshold, pearson correlation, optimization method genetic, toll equal 0.002 and box size 5x5."""))

In [57]:
dir_out = os.path.join('/','media','arturo','T9','Data','Italy','Satellite','5_DOWN')
DOWN_out = os.path.join(dir_out,f'Italy_DOWN_CHIRPS_1dy_2002_2023_npix_5_thr_1_acf_mar_genetic_pearson.nc')
print(f'Export Data to {DOWN_out}')
DOWN_xr.to_netcdf(DOWN_out)

Export Data to /media/arturo/T9/Data/Italy/Satellite/5_DOWN/Italy_DOWN_CHIRPS_1dy_2002_2023_npix_5_thr_1_acf_mar_genetic_pearson.nc
